The following notebook presents a CFD study of a flow over cylinder at Re - 500, using Watertight meshing and Fluent solver -- using Python scripting in ANSYS PyFluent 

**Importing the required files**

In [30]:
import ansys.fluent.core as pyfluent
from matplotlib import pyplot as plt
from matplotlib import image
%matplotlib inline

**Starting the meshing session**

In [31]:
# Open a meshing session------------------------------
meshing_session = pyfluent.launch_fluent(precision = "double", processor_count = 4, mode = "meshing", show_gui = True)
workflow = meshing_session.workflow
meshing = meshing_session.meshing

pyfluent.launcher WARNING: PyFluent Watchdog did not initialize correctly, proceeding without it...


Setting the Watertight meshing workflow

In [32]:
# The meshing workflow is selected as the "Watertight Meshing Workflow"
workflow.InitializeWorkflow(WorkflowType = r'Watertight Geometry')
# The global scale setting
meshing.GlobalSettings.LengthUnit.set_state(r'mm')
meshing.GlobalSettings.AreaUnit.set_state(r'mm^2')
meshing.GlobalSettings.VolumeUnit.set_state(r'mm^3')

Importing geometry

In [33]:
workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'Project_4_FlowCylinder_Re500.scdoc',})
workflow.TaskObject['Import Geometry'].Execute()

True

Local sizing - adding the BOIs in the geometry

In [34]:
#local sizing- setting up Body of Infulence for refining the mesh  ------------------------------------------
# 1st BOI
workflow.TaskObject['Add Local Sizing'].Arguments.set_state({r'AddChild': r'yes',r'BOIExecution': r'Body Of Influence',r'BOIFaceLabelList': \
                                                             [r'boi_1-solid'],r'BOISize': 0.5,})
workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate()   # executing the BOI - 1
# 2nd BOI

workflow.TaskObject['Add Local Sizing'].Arguments.set_state({r'AddChild': r'yes',r'BOIExecution': r'Body Of Influence',r'BOIFaceLabelList': \
                                                             [r'boi_2-solid'],r'BOISize': 2,})
workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate()  # executing the BOI - 2

# 3rd BOI

workflow.TaskObject['Add Local Sizing'].Arguments.set_state({r'AddChild': r'yes',r'BOIExecution': r'Body Of Influence',r'BOIFaceLabelList': \
                                                             [r'boi_3-solid'],r'BOISize': 4,})
workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate()  # executing the BOI - 3


True

The surface meshing

In [35]:
# -------Surface mesh --- Cells per gap, Curvature Normal angle, max size, min size ---------------------------------------
workflow.TaskObject['Generate the Surface Mesh'].Arguments.set_state({r'CFDSurfaceMeshControls': {r'CellsPerGap': 4,r'CurvatureNormalAngle': \
                                                                                                  10,r'MaxSize': 10, \
                                                                                                  r'MinSize': 4,},})
workflow.TaskObject['Generate the Surface Mesh'].Execute()


True

Describe geometry task

In [36]:
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=False)
workflow.TaskObject['Describe Geometry'].Arguments.set_state({r'SetupType': r'The geometry consists of only fluid regions with no voids',})
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=True)
workflow.TaskObject['Describe Geometry'].Execute()

True

Update boundaries

In [37]:

workflow.TaskObject['Update Boundaries'].Execute()
workflow.TaskObject['Update Regions'].Execute()

True

Add Boundary layers

In [38]:
# using last ratio method, changing the number of layers to 5, first height - 0.5 mm, transition ratio - 0.2, other setting default
workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'FirstHeight': 0.25,r'LocalPrismPreferences': \
                                                                {r'Continuous': r'Stair Step',},r'NumberOfLayers': 10, \
                                                                r'OffsetMethodType': r'last-ratio',r'TransitionRatio': 0.2,})
workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate()

# using smooth transition method, changing the number of layers to 5, keeping the other settings ad default
#workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'LocalPrismPreferences': {r'Continuous': r'Stair Step',},r'NumberOfLayers': 5,})
#workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate()

True

Generate the volume mesh

In [39]:
# poly hexcore as volume fill with method, keeping the minimum size default, maximum size as 4 mm
workflow.TaskObject['Generate the Volume Mesh'].Arguments.set_state({r'VolumeFill': r'poly-hexcore', \
                                                                     r'VolumeFillControls': {r'HexMaxCellLength': 10, \
                                                                                             r'HexMinCellLength': 4,r'PeelLayers': 2,},})
workflow.TaskObject['Generate the Volume Mesh'].Execute()

True

In [40]:
meshing_session.tui.file.write_mesh("Project_4_FlowCylinder_Re500")

In [41]:
meshing_session.exit()